# US15 - Custo Mensal de Água em Cada Parque Relacionado com a Sua Dimensão

## 1. Introdução

## 2. Código e Resultados

In [ ]:
# importação das bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statistics as st
import numpy as np
from scipy import stats

In [ ]:
# leitura do ficheiro com o consumo de água
data = pd.read_csv('water_consumption.csv', sep=';', skiprows = 1, names=['Park', 'Year', 'Month', 'Day', 'Consumption'])

data['Month'] = data['Month'].astype(int)
data['Year'] = data['Year'].astype(int)
data['Consumption'] = data['Consumption'].astype(str).str.replace(',', '.').astype(float)

## 3. Análise e Interpretação dos Resultados

# US16 - Determinar a Melhor Linha que se Ajusta aos Dados Obtidos na US14

## Introdução

## Código e Resultados

In [ ]:
# importação das bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# leitura do ficheiro
data = pd.read_csv('EquipmentUsed.csv', sep=';', skiprows = 1, names=['Equipment'])

## Análise e Interpretação de Resultados

## Contribuição dos Elementos do Grupo

**Diogo Pereira** = 35%

**Diogo Veiros** = 35%

**José Alves** = 15%

**Tiago Sampaio** = 15%